In [45]:
import pandas as pd

textblob_sentiment_output= pd.read_csv("textblob_sentiment_output.csv")
vader_sentiment_output= pd.read_csv("vader_sentiment_output.csv")
bert_sentiment_output= pd.read_csv("bert_sentiment_output.csv")

In [46]:
#as the merge on sentiment label was case sentsitive and giving zero output, we changed the case to lower case
bert_sentiment_output['sentiment_label'] = bert_sentiment_output['sentiment_label'].str.lower() 

In [47]:
# Merge all three on 'product id' and 'sentiment label'
merged = vader_sentiment_output.merge(
    textblob_sentiment_output[['product_id', 'sentiment_label']],
    on=['product_id', 'sentiment_label']
).merge(
    bert_sentiment_output[['product_id', 'sentiment_label']],
    on=['product_id', 'sentiment_label']
)

In [48]:
#m1=pd.merge(vader_sentiment_output, textblob_sentiment_output, on=['sentiment_label','product_id',], how='inner')
#m0=pd.merge(m1, bert_sentiment_output, on=['sentiment_label','product_id',], how='inner') 
merged

,star_rating,product_title,review_body,review_date,verified_purchase,helpful_vote,product_id,user_id,sentiment_score,sentiment_label
0,5.0,Cash money every time,[[VIDEOID:18bf6b3028e5267bf8a1ad02be5df64e]] E...,2023-02-16,True,5,B0BNVT9LJH,AH3QH7KS6OQDFEEHIPRDDHRN3ASA,0.9062,positive
1,5.0,Five Stars,"GOOD PRODUCT, GOOD PRICE",2016-02-17,False,0,B010EP1FR2,AGJDTRPHRL476AYCRQSOKVM62WZQ,0.7003,positive
2,5.0,Excellent product and makes VR cable managemen...,Excellent product and makes VR cable managemen...,2018-05-11,True,0,B01M6EB8DM,AFERUQZOZKCUJZ3WGUBHXXA63D7A,0.7712,positive
3,5.0,Strong protection,Very well,2019-12-31,True,0,B01IBE55TM,AECPW52RIYFWOGWV2NR7DUEA73QA,0.3384,positive
4,1.0,I hate to give a bad review but this case was ...,"Sorry, I hate to give a bad review but this ca...",2016-01-30,True,0,B015TDECF6,AGG53G3HBVHQQUZ77NBYP32OPDVA,-0.5789,negative
...,...,...,...,...,...,...,...,...,...,...
57,2.0,Screen Quality Looks OEM,UPDATE: I gifted this to my nephew for Christm...,2019-12-11,True,0,B07X9TNHM9,AGYBEEDJKBNN5O2Y3V55YEGIKENA,0.3499,positive
58,5.0,Five Stars,Don't have a Galaxy but my friend is enjoying ...,2015-04-06,False,0,B00J02VPVM,AF6OEDUJQQ5MVKQXZ757MSWG227A,0.9313,positive
59,1.0,*WARNING!* screen shattered from a 2 foot fall...,I bought this case because my other clear case...,2021-10-29,True,3,B08MS4626W,AEGYMK7G3OCOMO7ZT6XYKSYNW22A,-0.3021,negative
60,5.0,I really like these bands,I really like these bands. They're not fancy b...,2016-04-23,True,0,B013XFHV1E,AEXYMOHZLDBTLH3CXIECZ4SWXQAA,0.0942,positive


In [49]:
#merged.filter(merged['star_rating'] > 3 and merged['sentiment_label']=='positive')
merged=merged[((merged['star_rating'] > 3) & (merged['sentiment_label'] == 'positive')) | 
       (merged['star_rating'] < 3) & (merged['sentiment_label'] == 'negative')]

In [75]:
merged.to_csv("merged.csv", index=False)

In [71]:
# n- gram analysis

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

def generate_N_grams(text,ngram=2):
  text = text.lower()
  text = re.sub(r'<.*?>', '', text) # Remove HTML
  text = re.sub(r'[^a-z\s]', '', text) # Remove punctuation
  words=[word for word in text.split(" ") if word not in set(stopwords.words('english'))]  
  print("Sentence after removing stopwords:",words)
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
#sample!
#generate_N_grams("The sun rises in the east",2)
from collections import defaultdict


positiveValues= defaultdict(int)

#get the count of every word in both the columns of df_train and df_test dataframes where sentiment="positive"
for text in merged[merged.sentiment_label=="positive"].review_body:
  for word in generate_N_grams(text):
    positiveValues[word]+=1

df_positive=pd.DataFrame(sorted(positiveValues.items(),key=lambda x:x[1],reverse=True))

Sentence after removing stopwords: ['videoidbfbebfaadbedfe', 'esr', 'makes', 'best', 'cases', 'case', 'durable', 'worked', 'wonderfully', '', 'years', 'phone', 'course', 'revisit', 'well', 'another', 'ear', 'case', 'one', 'feels', 'sturdy', 'kickstand', 'gives', 'several', 'viewing', 'angles', 'horizontal', 'well', 'additional', 'vertical', 'viewing', 'angle', 'ill', 'always', 'come', 'back', 'get', 'esr', 'cases', 'phone', 'get']
Sentence after removing stopwords: ['good', 'product', 'good', 'price']
Sentence after removing stopwords: ['excellent', 'product', 'makes', 'vr', 'cable', 'management', 'dream', '', 'getting', 'tangled', 'cable', 'quality', 'product', 'good', 'plenty', 'included', 'pulleys', 'doublesided', 'adhesive', 'get', 'setup', 'works', 'room']
Sentence after removing stopwords: ['well']
Sentence after removing stopwords: ['case', 'perfect', '', 'cute', 'durable']
Sentence after removing stopwords: ['well', 'constructed', 'leather', 'case', 'designed', 'mophie', 'juice

In [73]:
negativeValues= defaultdict(int)

#get the count of every word in both the columns of df_train and df_test dataframes where sentiment="negative"
for text in merged[merged.sentiment_label=="negative"].review_body:
  for word in generate_N_grams(text):
    negativeValues[word]+=1

df_negative=pd.DataFrame(sorted(negativeValues.items(),key=lambda x:x[1],reverse=True))

Sentence after removing stopwords: ['sorry', 'hate', 'give', 'bad', 'review', 'case', 'really', 'cheaply', 'made']
Sentence after removing stopwords: ['thing', 'worked', 'ok', 'couple', 'week', 'barely', 'charges', 'watch', '', 'overnight', 'cant', 'tell', 'charging', 'power', 'weak', 'watch', 'doesnt', 'stay', 'magnet', 'also', 'time', 'put', 'watch', 'magnet', 'doesnt', 'begin', 'charging', 'unplugplugin', 'charger', 'number', 'times', 'starts', 'chargingvery', 'poor', 'quality', 'product']
Sentence after removing stopwords: ['alert', 'screen', 'protector', 'fit', 'iphone', 'long', 'also', 'camera', 'protector', 'match', 'either']
Sentence after removing stopwords: ['bought', 'case', 'clear', 'cases', 'kept', 'turning', 'yellow', 'makes', 'look', 'bad', 'however', 'cases', 'protected', 'phone', 'dozens', 'dozens', 'drops', 'never', 'single', 'scratch', 'phonewell', 'case', 'didnt', 'turn', 'yellow', 'like', 'said', 'wouldnt', 'certainly', 'didnt', 'provide', 'protection', 'phone', 'o

In [74]:
print(df_negative.head(4))
print(df_positive.head(20))

             0  1
0   didnt turn  2
1  turn yellow  2
2   sorry hate  1
3    hate give  1
                   0  1
0   highly recommend  3
1         phone case  3
2          phone get  2
3       product good  2
4         good price  2
5       would highly  2
6     recommend case  2
7        case anyone  2
8      youre looking  2
9           easy set  2
10       works great  2
11        case great  2
12        hold phone  2
13    protects phone  2
14        fit easily  2
15             good   2
16          far good  2
17                    2
18           iphone   2
19    happy purchase  2
